<a href="https://colab.research.google.com/github/nshamid/HO1_MLOps_NabilahShamid/blob/main/HO1_MLOps_NabilahShamid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Source

In [1]:
# Import libraries
import requests
import zipfile
import os
import pandas as pd
import numpy as np

In [2]:
# Dataset diunggah di Github untuk kemudahan akses
dataset_url = 'https://github.com/nshamid/HO1_MLOps_NabilahShamid/raw/refs/heads/main/data_scientist_jobstreet_scraped_v2.zip'

# Unduh file ZIP
response = requests.get(dataset_url)
with open('dataset.zip', 'wb') as f:
    f.write(response.content)

print("ZIP file berhasil diunduh!")

ZIP file berhasil diunduh!


In [3]:
# Mengecek nama file
with zipfile.ZipFile('dataset.zip', 'r') as zip_ref:
    file_list = zip_ref.namelist()
    print("Isi file di dalam ZIP:")
    for f in file_list:
        print(f)

    # Ekstrak semua file
    zip_ref.extractall()

print("ZIP file berhasil diekstrak!")

Isi file di dalam ZIP:
data_scientist_jobstreet_scraped_v2.csv
ZIP file berhasil diekstrak!


In [4]:
file_name = 'data_scientist_jobstreet_scraped_v2.csv'

# Load CSV ke DataFrame
df = pd.read_csv(file_name)

In [5]:
print(df.head())

   Unnamed: 0    job_id                              job_title  \
0           0  72761527                          Data Engineer   
1           1  72787241         Machine Learning Engineer (AI)   
2           2  72866732               Senior Risk/Data Analyst   
3           3  72851872  Senior Data Engineer (Hybrid Working)   
4           4  72526811        Data Scientist (Hybrid Working)   

                             company  \
0          ANHSIN TECHNOLOGY SDN BHD   
1            Accordia Global Sdn Bhd   
2  Toyota Capital Malaysia Sdn. Bhd.   
3                               SEEK   
4              SEEK Asia (JobStreet)   

                                        descriptions          location  \
0  Design, develop, and maintain scalable and rob...      Kuala Lumpur   
1  Design, develop, and deploy machine learning m...  Shah Alam/Subang   
2  Analyse data to better understand potential ri...          Petaling   
3  Design, development, testing, and operation of...      Kuala Lu

In [6]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 606 entries, 0 to 605
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Unnamed: 0    606 non-null    int64 
 1   job_id        606 non-null    int64 
 2   job_title     606 non-null    object
 3   company       606 non-null    object
 4   descriptions  588 non-null    object
 5   location      606 non-null    object
 6   category      606 non-null    object
 7   subcategory   606 non-null    object
 8   type          606 non-null    object
 9   salary        198 non-null    object
dtypes: int64(2), object(8)
memory usage: 47.5+ KB
None


# Prepocessing

In [7]:
!pip install pandas nltk Sastrawi nlp-id

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.3/484.3 kB 40.3 MB/s eta 0:00:00
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=c236664228d84f75376a2edccbcb3207095ffc335b095d4306ea2cb91032791b
  Stored in directory: /root/.cache/pip/wheels/40/b3/0f/a40dbd1c6861731779f62cc4babcb234387e11d697df70ee97
Successfully built wget
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.33.2
    Uninstalling huggingface-hub-0.33.2:
      Successfully uninstalled huggingface-hub-0.33.2


In [8]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [9]:
import pandas as pd
import re
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nlp_id.lemmatizer import Lemmatizer

# Inisialisasi stemmer dari Sastrawi
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Inisialisasi lemmatizer dari nlp_id
lemmatizer = Lemmatizer()

print("Library berhasil diimpor dan tools preprocessing diinisialisasi.")

Library berhasil diimpor dan tools preprocessing diinisialisasi.


## Menerapkan prepoccessing pada kolom
1. job_title (Judul Pekerjaan)

2. descriptions (Deskripsi Pekerjaan)

In [10]:
def clean_noise(text):
  # Menghapus semua tag HTML
  text = re.sub(r'<.*?>', '', text)
  # Menghapus URL
  text = re.sub(r'https?://\\S+|www\\.\\S+', '', text)
  # Menghapus Hashtag
  text = re.sub(r'#\\w+', '', text)
  # Menggunakan re.sub(r'[^\w\s]', '', text)
  text = re.sub(r'[^\w\s]', '', text)
  # Menghapus Angka
  text = re.sub(r'\\d+', '', text)
  # Menghapus spasi berlebih
  text = re.sub(r'\\s+', ' ', text).strip()
  return text

def remove_stopwords(text):
  list_stopwords = stopwords.words('indonesian')
  tokens = text.split()
  tokens_without_stopwords = [word for word in tokens if word not in list_stopwords]
  text = ' '.join(tokens_without_stopwords)
  return text

def cleaning_pipeline(text):
  if not isinstance(text, str):
      return text

  text = text.lower() # 1. Lowercasing
  text = clean_noise(text) # 2. Menghapus Noise
  text = remove_stopwords(text) # 3. Menghapus Stopwords
  text = stemmer.stem(text) # 4. Stemming
  return text

def normalize_categorical_string(text):
  if not isinstance(text, str):
      return text
  return text.lower().strip()

print("Fungsi-fungsi preprocessing telah didefinisikan (termasuk perbaikan regex untuk spasi).")

Fungsi-fungsi preprocessing telah didefinisikan (termasuk perbaikan regex untuk spasi).


In [11]:
# Muat dataset dari file CSV
try:
    # Menghapus kolom 'Unnamed'
    if 'Unnamed: 0' in df.columns:
        df = df.drop('Unnamed: 0', axis=1)
        print("\nKolom 'Unnamed: 0' berhasil dihapus.")

    print("\nMenerapkan cleaning pipeline (full cleaning) ke kolom teks bebas:")

    print("- Kolom 'job_title'...")
    df['cleaned_job_title'] = df['job_title'].apply(cleaning_pipeline)

    print("- Kolom 'descriptions'...")
    df['cleaned_descriptions'] = df['descriptions'].apply(cleaning_pipeline)

    print("\nMenerapkan normalisasi ringan ke kolom string kategori:")
    columns_to_normalize_lightly = ['company', 'location', 'category', 'subcategory', 'type']
    for col in columns_to_normalize_lightly:
        if col in df.columns:
            df[f'normalized_{col}'] = df[col].apply(normalize_categorical_string)
            print(f"- Kolom '{col}' dinormalisasi ke 'normalized_{col}'.")
        else:
            print(f"- Peringatan: Kolom '{col}' tidak ditemukan.")

    print("\nDataFrame head setelah pembersihan dan normalisasi (kolom relevan):")
    # Menampilkan beberapa kolom original dan yang sudah dibersihkan/dinormalisasi
    print(df[['job_title', 'cleaned_job_title',
              'descriptions', 'cleaned_descriptions',
              'company', 'normalized_company',
              'location', 'normalized_location',
              'category', 'normalized_category',
              'subcategory', 'normalized_subcategory',
              'type', 'normalized_type']].head())

    # Tampilkan kembali info DataFrame untuk melihat kolom baru
    print("\nDataFrame info setelah penambahan kolom bersih/normalisasi:")
    print(df.info())

except FileNotFoundError:
    print("Error: File 'data_scientist_jobstreet_scraped_v2.csv' tidak ditemukan. Pastikan file sudah diunggah.")
except KeyError as e:
    print(f"Error: Kolom {e} tidak ditemukan di DataFrame. Pastikan nama kolom sudah benar.")
except Exception as e:
    print(f"Terjadi kesalahan saat memuat atau memproses data: {e}")


Kolom 'Unnamed: 0' berhasil dihapus.

Menerapkan cleaning pipeline (full cleaning) ke kolom teks bebas:
- Kolom 'job_title'...
- Kolom 'descriptions'...

Menerapkan normalisasi ringan ke kolom string kategori:
- Kolom 'company' dinormalisasi ke 'normalized_company'.
- Kolom 'location' dinormalisasi ke 'normalized_location'.
- Kolom 'category' dinormalisasi ke 'normalized_category'.
- Kolom 'subcategory' dinormalisasi ke 'normalized_subcategory'.
- Kolom 'type' dinormalisasi ke 'normalized_type'.

DataFrame head setelah pembersihan dan normalisasi (kolom relevan):
                               job_title                    cleaned_job_title  \
0                          Data Engineer                        data engineer   
1         Machine Learning Engineer (AI)         machine learning engineer ai   
2               Senior Risk/Data Analyst              senior riskdata analyst   
3  Senior Data Engineer (Hybrid Working)  senior data engineer hybrid working   
4        Data Scientist 

In [12]:
if 'df' in locals() and not df.empty:
    # Simpan ke CSV
    csv_filename = 'cleaned_job_data.csv'
    df.to_csv(csv_filename, index=False)
    print(f"\nData bersih berhasil disimpan ke file CSV: '{csv_filename}'")

    # Simpan ke JSON
    json_filename = 'cleaned_job_data.json'
    df.to_json(json_filename, orient='records', indent=4)
    print(f"Data bersih berhasil disimpan ke file JSON: '{json_filename}'")
else:
    print("DataFrame kosong atau tidak ditemukan. Pastikan langkah memuat dan memproses data berhasil.")


Data bersih berhasil disimpan ke file CSV: 'cleaned_job_data.csv'
Data bersih berhasil disimpan ke file JSON: 'cleaned_job_data.json'
